In [2]:
#Import dependencies
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

#To perform Hot Encoding
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense

#To trasnform data
from datetime import datetime as dt

Using TensorFlow backend.


In [3]:
#Initial RMA dataframe cleaning (check duplicates, drop NaN for critical fields, and filter for "RMA Defective Item")
df1 = pd.read_csv('data_proj3.csv')
print("original shape: ", df1.shape)
print("duplicated rows: ", df1[df1.duplicated()].shape[0])
df1 = df1.dropna(subset = ['RMA Create Date', "As Built2 Ship Date"])
print("new shape: ", df1.shape)
df1['RMA Item Transaction Type'].value_counts()

/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


original shape:  (198768, 39)
duplicated rows:  0
new shape:  (180314, 39)


RMA Defective Item           83503
RMA Replacement Item         74409
Defective item for FCO        9660
Replacement item for FCO      7717
Actual Defective Received     4810
Defective Actual for FCO       214
RMA Conversion Defective         1
Name: RMA Item Transaction Type, dtype: int64

In [4]:
#Filter for defectives only and re-title for concatenation
df1 = df1.loc[df1['RMA Item Transaction Type'] == 'RMA Defective Item']
print("defectives only shape: ", df1.shape)
df1 = df1.rename(columns={"SR Customer ID": "Installed At - Key", "Product ID": "IB Material - Key", "As Built2 Ship Date": "Shipped Day"})
df1 = df1[['IB Material - Key', "Installed At - Key", 'Serial Number', 'Shipped Day', "RMA Create Date", "RMA Item Transaction Type"]]
df1.isnull().sum()

defectives only shape:  (83503, 39)


IB Material - Key            0
Installed At - Key           0
Serial Number                0
Shipped Day                  0
RMA Create Date              0
RMA Item Transaction Type    0
dtype: int64

In [5]:
#Initial Install Base(IB) dataframe cleaning (check duplicates, drop NaN for key fields) 
df2 = pd.read_csv('IB_data_ship_date.csv')
print("original shape: ", df2.shape)
print("duplicated rows: ", df2[df2.duplicated()].shape[0])
df2 = df2.drop_duplicates()
df2 = df2.dropna(subset=['Serial Number'])
print("new shape: ", df2.shape)
df2.isnull().sum()

original shape:  (1337779, 4)
duplicated rows:  12930
new shape:  (1324849, 4)


IB Material - Key     0
Installed At - Key    0
Serial Number         0
Shipped Day           0
dtype: int64

In [6]:
df2.head()

,IB Material - Key,Installed At - Key,Serial Number,Shipped Day
0,MXMPC4E3D32XGESFPP,100000596,CADX0717,2016-06-24
1,BT8A78CH2-I02,100053925,HC15180092,2016-06-30
2,BT8A78CH2-I02,100053925,HC15230332,2016-05-31
3,QFX5100-96S-AFO,100135543,VB3715300091,2016-06-30
4,EX4300-48T-AFI,100147020,PE3714140135,2016-11-11


In [7]:
#Concatenate for full DF
df = pd.concat([df1, df2], keys=['IB Material - Key', "Installed At - Key", 'Serial Number', 'Shipped Day', "RMA Item Transaction Type"])
df.reset_index(inplace=True)
df = df.drop(['level_0', 'level_1'], axis=1)
print("Final Shape: ", df.shape)
print(df.isnull().sum())

Final Shape:  (1408352, 6)
IB Material - Key                  0
Installed At - Key                 0
RMA Create Date              1324849
RMA Item Transaction Type    1324849
Serial Number                      0
Shipped Day                        0
dtype: int64


In [8]:
#Replace non-defective RMA Create Dates with today's date
df['RMA Create Date'].fillna(dt.today(), inplace=True)
df['RMA Item Transaction Type'].fillna(0, inplace=True)
df['RMA Item Transaction Type'].replace("RMA Defective Item", 1, inplace=True)
df.head(5)

,IB Material - Key,Installed At - Key,RMA Create Date,RMA Item Transaction Type,Serial Number,Shipped Day
0,81Y8237,100034495,3/20/2017,1,233G04B,10/12/2012
1,81Y8349,101400565,1/31/2018,1,231S01X,7/18/2011
2,95Y0510,101183390,4/12/2017,1,EM0711427457,10/29/2013
3,95Y0510,101183390,4/27/2017,1,EM0711427505,12/13/2012
4,95Y0510,101183390,4/27/2017,1,EM0711427426,12/13/2012


In [9]:
df = df.drop(10837).reset_index(drop=True)

In [10]:
### DO NOT RE-RUN... TAKES FOREVER
### Results saved in next cell

# lifespan = []
# for row in df.index:
#     print(row, "of", len(df.index))
#     try:
#         life = dt.strptime(df.iloc[row]['RMA Create Date'], '%m/%d/%Y') - dt.strptime(df.iloc[row]['Shipped Day'], '%m/%d/%Y')
#         lifespan.append(life.days)
#     except TypeError:
#         try:
#             life = df.iloc[row]['RMA Create Date'] - dt.strptime(df.iloc[row]['Shipped Day'], '%Y-%m-%d')
#             lifespan.append(life.days)
#         except:
#             print(row, " has error")
#             life = 'NaN'
#             lifespan.append(life)
#         else: 
#             pass


In [54]:
### Pull CSV
df = pd.read_csv('Model_Data.csv')

### Previous code to save CSV
# df['Lifespan(days)'] = pd.DataFrame(lifespan)
# print("df shape: ", df.shape)
# df = df.rename(columns={'RMA Item Transaction Type': 'Defective'})
# df.to_csv('Model_Data.csv')

/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [17]:
#####----------------- PLAYING -------------

,Unnamed: 0,IB Material - Key,Installed At - Key,RMA Create Date,Defective,Serial Number,Shipped Day,Lifespan(days)
0,0,81Y8237,100034495,3/20/2017,1,233G04B,10/12/2012,1620.0
1,1,81Y8349,101400565,1/31/2018,1,231S01X,7/18/2011,2389.0
2,2,95Y0510,101183390,4/12/2017,1,EM0711427457,10/29/2013,1261.0
3,3,95Y0510,101183390,4/27/2017,1,EM0711427505,12/13/2012,1596.0
4,4,95Y0510,101183390,4/27/2017,1,EM0711427426,12/13/2012,1596.0


In [ ]:
df['IB Material - Key'] = df['IB Material - Key'].astype(str)

In [48]:
products = df['IB Material - Key'].value_counts()
top_products = products[:100].tolist()
top_products_df = df.loc[df['IB Material - Key'].isin(top_products)]

print('top products shape: ', top_products_df.shape)

TypeError: '<' not supported between instances of 'int' and 'str'

In [14]:
print(top_products)

[56824, 48426, 43899, 35290, 33918, 31952, 27606, 26499, 23212, 23158, 21944, 20813, 20219, 20158, 18396, 18293, 18230, 18206, 17488, 17101, 16587, 14357, 14086, 12949, 12633, 12238, 11697, 11142, 10855, 10801, 10484, 10375, 10287, 10227, 10200, 10174, 10006, 9850, 9841, 9132, 8958, 8903, 8808, 8629, 8487, 8310, 8171, 8092, 7820, 7787, 7579, 7240, 7229, 7017, 6984, 6945, 6736, 6639, 6509, 6217, 6169, 6138, 5861, 5638, 5506, 5484, 5233, 5222, 5175, 5075, 5031, 5004, 4970, 4808, 4806, 4734, 4727, 4602, 4526, 4331, 4232, 4226, 4172, 3993, 3903, 3881, 3847, 3829, 3632, 3623, 3557, 3436, 3385, 3335, 3280, 3249, 3231, 3151, 3044, 3040]


In [55]:
df = df.drop(['Serial Number', 'RMA Create Date', 'Shipped Day'], axis=1)
df = df.dropna(subset=['Lifespan(days)'])
print("final dataframe shape: ", df.shape)


final dataframe shape:  (1408350, 5)


In [56]:
model_df = pd.get_dummies(df['IB Material - Key'], sparse=True)
model_df['Lifespan(days)'] = df['Lifespan(days)']
print("dummy shape: ", df.shape)
model_df.head()

dummy shape:  (1408350, 5)


,81Y8237,81Y8349,95Y0510,95Y0512,ACX-MIC-16CHE1T1CE,ACX-MIC-6GE-CU-SFP,ACX-MIC4OC31OC12CE,ACX1000-DC,ACX1100-AC,ACX1100-DC,...,XENPAK-1XGE-LR,XENPAK-1XGE-SR,XFP-10G-CBANDT50ZR,XFP-10G-E-OC192IR2,XFP-10G-L-OC192SR1,XFP-10G-S,XFP-10G-Z-OC192LR2,XFP-10GE-LR,XFP-10GE-SR,Lifespan(days)
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1620.0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2389.0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1261.0
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1596.0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1596.0


In [57]:
y = df['Defective']
X = model_df

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)
model = LinearRegression()
model.fit(X_train, y_train)

In [ ]:
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

In [47]:
print("Unique clients: ", df['Installed At - Key'].nunique())
print("Unique products: ", df['IB Material - Key'].nunique())

Unique clients:  66511
Unique products:  2352


In [20]:
df.head(5)

,Unnamed: 0,IB Material - Key,Installed At - Key,Defective,Lifespan(days)
0,0,81Y8237,100034495,1,1620.0
1,1,81Y8349,101400565,1,2389.0
2,2,95Y0510,101183390,1,1261.0
3,3,95Y0510,101183390,1,1596.0
4,4,95Y0510,101183390,1,1596.0


In [29]:
y = df['Defective'].reshape(-1,1)
X = df['Lifespan(days)'].reshape(-1,1)
print(y.shape, X.shape)

(1408350, 1) (1408350, 1)


/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """Entry point for launching an IPython kernel.
/Users/yuh13/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  


In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [31]:
training_score = model.score(X_train, y_train)
testing_score = model.score(X_test, y_test)

print(f"Training Score: {training_score}")
print(f"Testing Score: {testing_score}")

Training Score: 0.24006505660854827
Testing Score: 0.23676432407084225


Object `%mprun` not found.
Object `%memit` not found.
